In [1]:
# import pandas as pd
# import numpy as np
# import re
# import os
# import string
import fn_module
from fn_module import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = fn_module.read_file("CSE508_Winter2023_Dataset_Processed_new")
print("File count = ", len(data))

keys = list(data.keys())

s = ""
df = []
for key in data:
    df.append(data[key])


File count =  1400


In [3]:
from collections import defaultdict

def create_inverted_index(documents):
    inverted_index = defaultdict(list)
    
    
    for doc_id, document in enumerate(documents):
        string1 = document.split()
        for word in range(len(string1)-1):
            bi_string = string1[word:word+2]
            bi = " ".join(bi_string)
            inverted_index[bi].append(doc_id)
    
    return inverted_index




In [4]:
import pickle
from collections import defaultdict

inverted_index = create_inverted_index(df)

def save_inverted_index(inverted_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(inverted_index, f)
        
def load_inverted_index(file_path):
    with open(file_path, 'rb') as f:
        inverted_index = pickle.load(f)
        
    return inverted_index

In [5]:
from collections import defaultdict

def create_positional_index(documents):
    pos_index = defaultdict(list)
    for no_1, doc in enumerate(documents):
        conv_lower = doc.lower()
        for no_2, token_no in enumerate(conv_lower.split()):
            if token_no not in pos_index:
                pos_index[token_no] = defaultdict(list)
            if no_2 not in pos_index[token_no]:
                pos_index[token_no][no_1] = []
            pos_index[token_no][no_1].append(no_2)
    return pos_index

pos_index = create_positional_index(df)


In [6]:
import pickle 
def save_positional_index(pos_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(pos_index, f)
        
def load_positional_index(file_path):
    with open(file_path, 'rb') as f:
        pos_index = pickle.load(f)
    return pos_index

In [29]:
def and_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
        else:
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
    return final_set, count_comparisons

def or_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            final_set.add(s_2[ptr_2])
            ptr_2 = ptr_2 + 1
        else:
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
        
    while(ptr_1 < len_1):
        final_set.add(s_1[ptr_1])
        ptr_1 += 1
        
    while(ptr_2 < len_2):
        final_set.add(s_2[ptr_2])
        ptr_2 += 1
        
    result = sorted(final_set)
        
    return result, count_comparisons

def ornot_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_1 = ptr_1 + 1
            ptr_2 = ptr_2 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
        else:
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
        
    while(ptr_1 < len_1):
        final_set.add(s_1[ptr_1])
        ptr_1 += 1
        
        
    result = sorted(final_set)
        
    return result, count_comparisons

def andnot_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_1 = ptr_1 + 1
            ptr_2 = ptr_2 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            final_set.add(s_2[ptr_2])
            ptr_2 = ptr_2 + 1
        else:
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
    return final_set, count_comparisons


In [25]:
def get_list(inverted_index , word):
    return inverted_index[word]

In [33]:
def retrieve_documents_bigram(bi_words, inverted_index, log = False):
    count = 0
    l1 = []

    if len(bi_words)>0:
        l1 = get_list(inverted_index,bi_words[0])
        if log:
            print("L1 :",l1)
        for i in range(1,len(bi_words)):
            l2 = get_list(inverted_index,bi_words[i])
            
            l1, count = and_operation(l1,l2)

            if log:
                print("L"+str(i+1),":",l2)
                print("Final Output after Performing AND :",l1)
                
    return l1,count

In [62]:
def retrieve_documents_positional(positional_index, terms, log = False):
    if len(terms) == 0:
        return []
    s1 = set(positional_index[terms[0]].keys())
    if log:
        print("L1 =",list(s1))
    
    for term in terms:
        if term in positional_index:
            s1 = set.intersection(s1,set(positional_index[term].keys()))
            if log:
                print("L2 =",set(positional_index[term].keys()))
                print("List after AND Operation =",list(s1))
    return list(s1)

In [68]:

t = int(input())
for _ in range(t):
    sentence = input()
    trans_sentence = fn_module.preprocess(sentence)
    # print("Processed sequence : ",trans_sentence)
    
    bi_words = []
    for word in range(len(trans_sentence)-1):
        string = trans_sentence[word] +" "+ trans_sentence[word+1]
        bi_words.append(string)
        
    # print("bi_word sequence : ",bi_words)

    l,count = retrieve_documents_bigram(bi_words, inverted_index)
    
    print("Number of documents retrieved for query "+str(_+1)+" using bigram inverted index :",len(l))
  
    print("Names of the documents retrieved for query "+str(_+1)+" using bigram inverted index :")
    
    if len(l)==0:
        print("NO DOCUMENT FOUND!!")
    else:
        doc_lst = []
        for val in l:
            doc_lst.append(keys[val])
        print(doc_lst)
    # print("Number of comparisons required for query "+str(_+1)+" using bigram inverted index :",count)


    pos_lst = retrieve_documents_positional(pos_index,trans_sentence)

    print("Number of documents retrieved for query "+str(_+1)+" using positional index :",len(pos_lst))
  
    print("Names of the documents retrieved for query "+str(_+1)+" using positional index :")
    
    if len(l)==0:
        print("NO DOCUMENT FOUND!!")
    else:
        doc_lst = []
        for val in pos_lst:
            doc_lst.append(keys[val])
        print(doc_lst)
    print("-"*150)
    

    

Number of documents retrieved for query 1 using bigram inverted index : 17
Names of the documents retrieved for query 1 using bigram inverted index :
['cranfield1220', 'cranfield0293', 'cranfield1381', 'cranfield0007', 'cranfield0040', 'cranfield0008', 'cranfield0043', 'cranfield0079', 'cranfield0080', 'cranfield0337', 'cranfield1264', 'cranfield1300', 'cranfield1205', 'cranfield0314', 'cranfield1211', 'cranfield0796', 'cranfield1278']
Number of documents retrieved for query 1 using positional index : 50
Names of the documents retrieved for query 1 using positional index :
['cranfield1284', 'cranfield0133', 'cranfield0261', 'cranfield0007', 'cranfield0008', 'cranfield0009', 'cranfield0525', 'cranfield1040', 'cranfield0272', 'cranfield1300', 'cranfield0024', 'cranfield0668', 'cranfield0796', 'cranfield0798', 'cranfield1188', 'cranfield0293', 'cranfield0294', 'cranfield0933', 'cranfield0040', 'cranfield0043', 'cranfield1324', 'cranfield1325', 'cranfield0053', 'cranfield1205', 'cranfield0

In [16]:
print(inverted_index['investigation aerodynamics'])
print(inverted_index['aerodynamics wing'])
print(inverted_index['wing slipstream'])
print(and_operation(inverted_index['investigation aerodynamics'],inverted_index['aerodynamics wing']))

[0]
[0]
[0]
{0}


In [17]:
# print(inverted_index.values())
print(inverted_index['investigation aerodynamics'])

[0]


In [55]:
# print(pos_index['boundary'].keys())
terms = ['boundary', 'layer', 'transition']
print(set(pos_index[terms[0]].keys()))

{1, 2, 3, 6, 7, 8, 11, 15, 16, 17, 20, 21, 22, 23, 33, 36, 39, 42, 44, 46, 48, 49, 52, 53, 54, 58, 61, 62, 70, 71, 72, 73, 75, 78, 79, 88, 93, 95, 100, 103, 104, 106, 110, 111, 116, 120, 124, 125, 126, 127, 130, 132, 133, 134, 139, 141, 144, 147, 148, 149, 154, 159, 162, 164, 167, 169, 172, 173, 178, 179, 187, 188, 190, 191, 194, 206, 208, 219, 221, 226, 239, 240, 241, 243, 253, 254, 255, 259, 260, 263, 264, 266, 268, 270, 271, 275, 281, 290, 291, 292, 293, 297, 298, 299, 302, 303, 304, 305, 306, 307, 308, 309, 313, 314, 315, 317, 319, 320, 321, 323, 324, 325, 326, 327, 328, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 351, 352, 354, 357, 358, 362, 363, 364, 365, 374, 375, 376, 377, 380, 381, 382, 387, 392, 396, 405, 412, 416, 421, 423, 424, 431, 434, 438, 447, 449, 451, 453, 456, 457, 458, 459, 460, 474, 475, 476, 477, 478, 480, 483, 486, 488, 492, 495, 503, 517, 524, 525, 526, 527, 528, 537, 539, 541, 546, 550, 552, 554, 558, 559, 561, 562, 563, 564